In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
class PowerFactorLoss_KPI ():
    self.block_values=[]
    
    def __init__(self, base_power, fluctuation_threshold, block_time, current_time):
        self.base_power=base_power
        self.fluctuation_threshold=fluctuation_threshold
        self.wall_clock=current_time
        self.block_time=block_time                # this is 30 mins but can be changed 
    
    
    def initialize (self, power_t, power_factor_t, current_time):
        self.block_counter=1
        self.block_average_power=power_t
        self.block_average_pf=power_factor_t
        self.block_start_time=current_time
        return
    
    
    def __power_fluctuation(self, power_t):
        if (abs(power_t-self.block_average_power)/self.block_average_power <= self.fluctation_threshold):
            return (0)
        else: 
            return (1)
    
    
    def __update_average_power (self, power_t):
        return ((self.block_average_power*self.block_counter + power_t)/(self.block_counter+1))
    
    
    def __update_average_power_factor (self, power_factor_t):
        return ((self.block_average_pf*self.block_counter + power_factor_t)/(self.block_counter+1))
        
        
    def __calculate_power_loss (self):
        n = len(self.block_values)
        kVAh_loss = np.sum([(a[1]/a[2]]-a[1]/self.base_power)*(a[0]/60.0) for a in self.block_values)
        kWAh_loss = kVAh_loss * np.sum([a[2]*(a[0]/60.0) for a in self.block_values])/(np.sum([a[0] for a in self.block_values])/60.0)
        kVArh_lag = np.sqrt(kVAh_loss**2 - kWAh_loss**2)
        kVArh_lead = np.sqrt(kVAh_loss**2 - (kVAh_loss*self.base_power)**2)
        cycle_power_factor = kWAh_loss / sqrt(kVAh_loss**2 + (kVArh_lag + kVArh_lead)**2)
                            
        return ({'kVAh_loss':kVAh_loss, 'kWAh_loss':kWAh_loss, 'cycle_power_factor':cycle_power_factor})
        
                 
    def check (self, power_t, power_factor_t, current_time):
        if self.__power_fluctuation (power_t):
            # store block time interval, average power, average pf                    
            # NOTE: Store time interval in minutes
            self.block_values=self.block_values+[current_time - self.block_start_time, self.block_average_power, self.block_average_pf]
            self.initialize (power_t, power_factor_t, current_time)
        else:
            self.__update_average_power (power_t)
            self.__update_average_power_factor (power_factor_t)
        
        if (current_time-self.wall_clock) >= self.block_time:
            return (self.__calculate_power_loss())
        else: return (0)
                 
        
            
            
            
        
        
    

In [ ]:
# Initialize the KPI

BASE_POWER, POWER_FLUCTUATION_ACCEPTED, TIME_BLOCK = 0.99, 0.05, 30     # Time is in minutes
PowerFactorLoss_KPI (BLOCK_POWER, POWER_FLUCTUATION_ACCEPTED, TIME_BLOCK, current_time)

PowerFactorLoss_KPI.initialize (instant_power, instant_power_factor, current_time)

...loop of the streaming data...

PowerFactorLoss_KPI.check(instant_power, instant_power_factor, current_time)

......

